## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Coihaique,CL,-45.5752,-72.0662,54.21,76,100,1.14,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,80.64,74,26,10.22,light rain
2,2,Iqaluit,CA,63.7506,-68.5145,-20.31,75,0,2.30,clear sky
3,3,Kodiak,US,57.7900,-152.4072,36.91,81,100,10.36,light rain
4,4,Mount Gambier,AU,-37.8333,140.7667,69.71,60,100,13.80,overcast clouds


In [4]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
suitable_temp = city_data_df.loc[((city_data_df["Max Temp"] >= 70) & (city_data_df["Max Temp"] <= 90))] 

In [6]:
suitable_temp.head(20)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,80.64,74,26,10.22,light rain
6,6,Hermanus,ZA,-34.4187,19.2345,70.81,89,31,9.51,scattered clouds
9,9,Avarua,CK,-21.2078,-159.7750,82.45,74,49,2.30,scattered clouds
10,10,Sawtell,AU,-30.3833,153.1000,79.18,50,75,13.80,broken clouds
11,11,Port Alfred,ZA,-33.5906,26.8910,70.61,86,16,3.98,light rain
12,12,Kahului,US,20.8947,-156.4700,82.42,66,20,6.91,few clouds
18,18,Hobart,AU,-42.8794,147.3294,70.27,61,75,5.75,broken clouds
22,22,Puerto Ayora,EC,-0.7393,-90.3518,78.33,81,25,4.38,scattered clouds
29,29,Buala,SB,-8.1450,159.5921,83.50,77,62,5.32,light rain
32,32,Dubbo,AU,-32.2500,148.6167,84.33,37,0,9.22,clear sky


In [7]:
suitable_temp.count()

City_ID                243
City                   243
Country                241
Lat                    243
Lng                    243
Max Temp               243
Humidity               243
Cloudiness             243
Wind Speed             243
Current Description    243
dtype: int64

In [8]:
# 4a. Determine if there are any empty rows.
# np.where(pd.isnull(suitable_temp))
suitable_temp.count().isnull().sum()


0

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = suitable_temp[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,80.64,light rain,-23.1203,-134.9692,
6,Hermanus,ZA,70.81,scattered clouds,-34.4187,19.2345,
9,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,
10,Sawtell,AU,79.18,broken clouds,-30.3833,153.1000,
11,Port Alfred,ZA,70.61,light rain,-33.5906,26.8910,
12,Kahului,US,82.42,few clouds,20.8947,-156.4700,
18,Hobart,AU,70.27,broken clouds,-42.8794,147.3294,
22,Puerto Ayora,EC,78.33,scattered clouds,-0.7393,-90.3518,
29,Buala,SB,83.50,light rain,-8.1450,159.5921,
32,Dubbo,AU,84.33,clear sky,-32.2500,148.6167,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,80.64,light rain,-23.1203,-134.9692,People ThankYou
6,Hermanus,ZA,70.81,scattered clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
9,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,Paradise Inn
10,Sawtell,AU,79.18,broken clouds,-30.3833,153.1000,The Bungalow Sawtell
11,Port Alfred,ZA,70.61,light rain,-33.5906,26.8910,The Halyards Hotel
12,Kahului,US,82.42,few clouds,20.8947,-156.4700,Maui Seaside Hotel
18,Hobart,AU,70.27,broken clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
22,Puerto Ayora,EC,78.33,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
29,Buala,SB,83.50,light rain,-8.1450,159.5921,Maringe Lagoon Lodge
32,Dubbo,AU,84.33,clear sky,-32.2500,148.6167,Quality Inn Dubbo International


In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count().isnull().sum()


0

In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))